In [14]:
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import requests
import re

# 1. Web scraping

First we extract the information (postal code, borough, neighborhood) in Wikipedia. 

The next function it'll be used for split information in three columns (['PostalCode', 'Borough', 'Neighborhood'])

In [ ]:
def get_data_in_cell(content):
    code = content.find('b').text
    neighborhood_borough = content.find('span').text
    pattern = re.compile(r"^([\w\s']+)\((.+)\)")
    borough = np.nan
    neighborhoods = np.nan
    if pattern.match(neighborhood_borough):
        matcher = pattern.search(neighborhood_borough)
        borough = matcher.group(1)
        neighborhoods = matcher.group(2)
        neighborhoods = neighborhoods.replace(" /", ",")
    return [code, borough, neighborhoods]

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text
soup = BeautifulSoup(page, 'lxml')

We show the data

In [18]:
table_html = (soup.find_all('table'))[0]
table_td = table_html.find_all('td')
list_for_df =list(map(get_data_in_cell, table_td))
df_web = pd.DataFrame(list_for_df, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df_web.dropna(inplace=True)
df_web

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')

In [11]:
merge_df = pd.merge(df_web, df_coordinates, left_on='PostalCode', right_on='Postal Code')
df = merge_df[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [12]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
